## Input data requirements

The input ice sheet model should be saved as a netCDF file. There are several requirements for the comparison:

### `Lithk` variable

The uploaded model to contain thickness data (the `lithk` variable) for the comparison.

### Rectangular grid

At time of writing, models *must* be defined on a rectangular X-Y grid in the ISMIP6 standard projected polar-stereographic space. (Note, NOT a lat-lon grid!) The ISMIP6 standard projection is defined [here](http://www.climate-cryosphere.org/wiki/index.php?title=ISMIP6_wiki_page). 


In [1]:
# set geospatial python3 kernel to provide needed packages
import os,sys
import glob as glob
import numpy as np
import h5py
import xarray as xr
import glob as glob
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from shapely.geometry import Point



import cartopy
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

# note: suppress numpy.dtype size changed warnings
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")

warnings.filterwarnings('ignore')

### Set model dir path, ice sheet region and start and end date

In [2]:
#Set the model data dir path
nc_filename='/home/jovyan/CmCt/notebooks/Gravimetry/lithk_GIS_JPL_ISSM_asmb.nc'

# Define the flag for the ice sheet region Greenland or Antarctica
icesheet = "Greenland" # Change to "Antarctica" Or "Greenland"

# Define start and end dates
start_date = np.datetime64('2000-01-01')
end_date = np.datetime64('2015-01-01')

### Select projection, shapefile and csv file for imbie

In [3]:
# Set shapefile path and projection and imbie csv_filebased on the ice sheet region
if icesheet == "Greenland":
    projection = "EPSG:3413"  # Greenland
    shapefile_path = "/home/jovyan/CmCt/notebooks/IMBIE/Greenland_Basins_PS_v1.4.2/Greenland_Basins_PS_v1.4.2.shp"
    csv_file_path = '/home/jovyan/CmCt/notebooks/IMBIE/imbie_greenland_2021_Gt.csv'
    
elif icesheet== "Antarctica":
    projection = "EPSG:3031"  # Antarctica
    shapefile_path = "/home/jovyan/CmCt/notebooks/IMBIE/ANT_Basins_IMBIE2_v1.6/ANT_Basins_IMBIE2_v1.6.shp"
    csv_file_path = '/home/jovyan/CmCt/notebooks/IMBIE/imbie_antarctica_2021_Gt.csv'
else:
    raise ValueError("Invalid iceshee value. Must be 'Greenland' or 'Antarctica'.")


### Caluclate  model mass balance for each basin and total mass cbalance for whole region

In [4]:
# Load the dataset(model data)
gis_ds = xr.open_dataset(nc_filename)
lithk = gis_ds['lithk']


# Interpolate lithk values at the start and end dates
lithk_start = lithk.interp(time=start_date).data.transpose().flatten()
lithk_end = lithk.interp(time=end_date).data.transpose().flatten()


# Calculate the difference
lithk_delta = lithk_end - lithk_start

# Replace NaN values with 0
lithk_delta[np.isnan(lithk_delta)] = 0


# Change Ice thickness unit from (m) to mass (kg) to gigatonnes(Gt)
# ice thickness*area* density of ice* 1e-12

rho_ice = 934 # (density of ice, kg/m^3)

#calculate area = x_resolution*y_resolution
x_coords = gis_ds['x'].values
y_coords = gis_ds['y'].values
x_resolution = abs(x_coords[1] - x_coords[0])
y_resolution = abs(y_coords[1] - y_coords[0])

lithk_delta = (lithk_delta * x_resolution*y_resolution)*rho_ice * 1e-12


# Create a list of Point geometries from coordinate grids
points = [Point(x, y) for x in x_coords for y in y_coords]

# Flatten lithk_delta to match the points list 
lithk_delta_flat = lithk_delta.flatten()

# Create DataFrame
lithk_df = pd.DataFrame({
    'geometry': points,
    'lithk_delta': lithk_delta_flat
})

# Convert DataFrame to GeoDataFrame
lithk_gdf = gpd.GeoDataFrame(lithk_df, geometry='geometry', crs=projection)


# Load basin shapefile 
basins_gdf = gpd.read_file(shapefile_path)


# Perform spatial join
# joined_gdf = gpd.sjoin(lithk_gdf, basins_gdf, how="inner", op='intersects')
joined_gdf = gpd.sjoin(lithk_gdf, basins_gdf, how="inner", predicate='intersects')


# Sum lithk_delta values by basin
basin_mass_change_sums = joined_gdf.groupby('index_right')['lithk_delta'].sum()
print('Basin_mass_change_sums =', basin_mass_change_sums)


# Sum all of the basin mass change
model_total_mass_balance= basin_mass_change_sums.sum()
print('Total_mass_change_sum =', model_total_mass_balance)


Basin_mass_change_sums = index_right
0   -126.337912
1   -153.186299
2   -113.911525
3   -144.087264
4   -187.140113
5   -247.869015
6   -457.155972
Name: lithk_delta, dtype: float64
Total_mass_change_sum = -1429.6881006840963


### Imbie mass balance

In [5]:
# Load the CSV file
mass_balance_data = pd.read_csv(csv_file_path)

# Adjust the column names based on the CSV content
date_column = 'Year'
mass_balance_column = 'Mass balance (Gt/yr)'

# Convert the 'Year' column to datetime format
mass_balance_data[date_column] = pd.to_datetime(mass_balance_data[date_column], format='%Y')

# Filter the data between the start and end dates
filtered_data = mass_balance_data[(mass_balance_data[date_column] >= start_date) & 
                                  (mass_balance_data[date_column] <= end_date)].copy()

# Convert Mass balance from Gt/yr to Gt/month
filtered_data.loc[:, 'Mass_balance_Gt'] = filtered_data[mass_balance_column] / 12

# Sum the total mass balance in the filtered range
imbie_total_mass_change_sum = filtered_data['Mass_balance_Gt'].sum()

# Print the results
print(f"Total Mass Balance from {start_date} to {end_date}: {imbie_total_mass_change_sum} Gt")


Total Mass Balance from 2000-01-01 to 2015-01-01: -3299.217333333333 Gt


### Calculate mass balance difference of imbie and model data

In [6]:
#Difference of imbie-model  mass change 
delta_masschange=imbie_total_mass_change_sum-model_total_mass_balance
print(f"Difference of imbie-model  mass change: {delta_masschange}")

Difference of imbie-model  mass change: -1869.5292326492367
